# VWR2A Simulator
This notebook illustrates how to use the simulator both for decoding existing hexadecimal VWR2A kernels to assembly, as well as writing your own kernels by writting your own assembly and generating it's bitstream. At the end, we develop a working kernel that adds two vectors together.

In [ ]:
# Imports
import pandas as pd
from random import randint
from src import *
from src.simulator import SIMULATOR

## ISAs for specialized slots
First, we set up objects for each specialized slot of the VWR2A (i.e. Load Store Unit, Reconfigurable Cells, etc.) and show some examples. For detailed descriptions of the assembly ISA ot the hexadecimal underlying, please visit the docs section.

### Loop Control Unit
This unit is prepared to control the loops of the code.

In [ ]:
# --------------------------------------------
#         Loop Control Unit (LCU)
# --------------------------------------------
lcu = LCU()

instr_list = ["NOP", "EXIT", "SADD R1, ZERO, LAST", "SADD R1, SRF(3), LAST", "SADD R1, 7, ONE", "SSUB SRF(4), SRF(4), SRF(4)", "JUMP 7, ONE", "BGEPD ZERO, ONE, 5"]

for instr in instr_list:
    _, _, imem_word = lcu.asmToHex(instr)
    print("ASM : " + instr + " --> Hex: " + imem_word.get_word_in_hex())

### Load-Store Unit
This unit is prepared to control the movement of data between the SPM and the VWRs.

In [ ]:
# --------------------------------------------
#             Load- Store Unit (LCU)
# --------------------------------------------
lsu = LSU()

instr_list = ["SADD R0, ONE, ONE/LD.VWR VWR_A", "SADD R0, SRF(5), ONE/SH.IL.UP", "SADD SRF(5), SRF(5), ONE/LD.VWR SRF"]

for instr in instr_list:
    _, _, imem_word = lsu.asmToHex(instr)
    print("ASM : " + instr + " --> Hex: " + imem_word.get_word_in_hex())

### Reconfigurable Cells
This units are prepared to make the computations as an ALU would do on a CPU.

In [ ]:
# --------------------------------------------
#         Reconfigurable Cells (RCs)
# --------------------------------------------
rc = RC()

instr_list = ["NOP", "SADD VWR_A, VWR_A, VWR_B", "SADD VWR_A, SRF(3), VWR_B", "LOR R0, RCB, MIN_INT", "MUL.FP R0, RCB, MIN_INT"]

for instr in instr_list:
    _, _, _, imem_word = rc.asmToHex(instr)
    print("ASM : " + instr + " --> Hex: " + imem_word.get_word_in_hex())

### Multiplexer Control Unit
This unit is prepared to take care of all the indexes of the SRF and VWR that are accesed for loads or stores.

In [ ]:
# --------------------------------------------
#      Multiplexer Control Unit (MXCU)
# --------------------------------------------
mxcu = MXCU()

instr_list = ["NOP", "SADD R1, ONE, LAST", "LOR R1, ONE, SRF(3)"]

for instr in instr_list:
    imem_word = mxcu.asmToHex(instr, -1, 0, 0, [0,0,0,0], 0)
    print("ASM : " + instr + " --> Hex: " + imem_word.get_word_in_hex())

## App Example
Now, let's see an example of a real program that adds two vectors.
For adding two vectors, you just need to add each element one by one until the end.
Let's assume the vectors have 128 elements, so they fit in one line of the SPM.
First, we store the values on the SPM.

## Generating code for kernels

### Process an existing kernel

Load an existing kernel (in the form of an excel sheet where each row is a clock cycle and each column is a specialized slot) and use the simulator to understand what is going on in each element at a given clock cycle. 

In [ ]:
kernel_path = "kernels/mf_q64_erosion/"
df = pd.read_csv(kernel_path + "instructions_hex.csv")
print("The instruction memory has {0} entries.".format(len(df)))
df.head()

Let's generate the assembly for this hexadecimal instructions so we understand better what is going on.

In [ ]:
sim = SIMULATOR()
sim.compileHexToAsm(kernel_path)

In [ ]:
df = pd.read_csv(kernel_path + "instructions_asm.csv")
print("The instruction memory has {0} entries.".format(len(df)))
df.head()

For example, let's make sure that the last instruction of the LCU is an EXIT. For this we need to know some information about the kernel. In the hexadecimal it is provided as the KMEM column. And we also extract the kernel_number from which line has the instruction.

In [ ]:
kernel_number = 1 # Asign a number for the kernel (coherent with the KMEM)
hex_word = "0x802b"
nInstr, _, _, _ = KMEM_WORD(hex_word=hex_word).decode_word()
print("Last instruction for LCU is: " + df.iloc[nInstr]['LCU'])

## Load a kernel

To load a kernel into the IMEM of the VWR2A we need to know some info about it.
 - The kernel number
 - How many and which columns it uses
 - How many instructions per column it has
 - The position where it starts in the IMEM
 - The postition in the SPM where the SRF initial values are

In [ ]:
kernel_path = "kernels/add_vectors/"
kernel_number = 1 # Kernel number (from 1 to 15)
column_usage = [True, False] # Columns to use
nInstrPerCol = 37 # Number of instructions per column
imem_add_start = 0 # Start address on imem for this kernel
srf_spm_addres = 0 # Line of the SPM with the initial data for the SRF

sim = SIMULATOR()
sim.kernel_config(column_usage, nInstrPerCol, imem_add_start, srf_spm_addres, kernel_number)

Now, let's generate assembly for it so we clearly see the adds.

In [ ]:
sim.compileHexToAsm(kernel_path)
df = pd.read_csv(kernel_path + "instructions_asm.csv")
print("The instruction memory has {0} entries.".format(len(df)))
df.head(10)

We can see that the same action, adding two elements, is performed one and once again. Let's handle that with a loop.

### Using loops
It's your time to play. Try to use a loop to reduce the number of instructions.
(You can think about it or check the solution in the provided assembly instructions version 2.)

## Running code

Now, let's run an application to see the outputs and check if the result is the expected.
We will use, the vectors addition example once again. So, let's load it.

In [ ]:
sim = SIMULATOR()

# --------------------------------------------
#               KERNEL CONFIGURATION
# --------------------------------------------
kernel_path = './kernels/add_vectors/'
kernel_number = 1 
column_usage = [True, False] 
nInstrPerCol = 6 
imem_add_start = 0 
srf_spm_addres = 0 
version="_v2"

sim.kernel_config(column_usage, nInstrPerCol, imem_add_start, srf_spm_addres, kernel_number)

Now, we need to populate the SPM with the values of our vectors.

In [ ]:
# --------------------------------------------
#                LOAD SPM DATA
# --------------------------------------------
# Load vector A
vector_A = [i for i in range(SPM_NWORDS)]
nline = 1
sim.setSPMLine(nline, vector_A)
# Load vector B
vector_B = [i for i in range(SPM_NWORDS)]
nline = 2
sim.setSPMLine(nline, vector_B)

sim.displaySPMLine(1)
sim.displaySPMLine(2)

Now, let's compile the assembly to hexadecimal since it's needed to run the code.

In [ ]:
# --------------------------------------------
#              COMPILE ASM TO HEX
# --------------------------------------------
sim.compileAsmToHex(kernel_path, kernel_number, version=version)

Finally, we load the kernel into the internal memory of the specialized units and run it.

In [ ]:
# --------------------------------------------
#                 LOAD KERNEL
# --------------------------------------------

# This needs the hex instructions, if you don't provide them, generate then compiling the asm
sim.kernel_load(kernel_path, version=version + "_autogen", kernel_number=kernel_number)

# --------------------------------------------
#               SIMULATE EXECUTION
# --------------------------------------------
show_lcu = []
show_srf = []
show_lsu = []
show_rcs = [[],[],[],[]]
show_mxcu = []
display_ops = [show_lcu, show_lsu, show_mxcu, show_rcs, show_srf]

sim.run(kernel_number, display_ops=display_ops)

Let's check that we have the correct output in the SPM line just by looking at it.

In [ ]:
sim.displaySPMLine(1)
sim.displaySPMLine(2)
sim.displaySPMLine(3)

We can check it more rigorously. We can define our function in python and check that the output matches the CGRA output.

In [ ]:
sim.displaySPMLine(3)
vwr2a_res = sim.getSPMLine(3)
errors_idx = []
for i in range(len(vector_A)):
    if vector_A[i] + vector_B[i] != vwr2a_res[i]:
        errors_idx.append(i)
if len(errors_idx) == 0:
    print("The result is correct!")
else:
    print("Oops, something went wrong. There are " + str(len(errors_idx)) + " errors.")
    print(errors_idx)


Now it's your time to play!

In [ ]:
#Let's do it!

# Appendix
For the rest of the examples, in order to run them some info for the kernel configuration is needed. It can be decoded from the hexadecimal words in the column KMEM.

### Exit

In [ ]:
# --------------------------------------------
#               KERNEL CONFIGURATION
# --------------------------------------------
kernel_path = './kernels/exit/' 
kernel_number = 1
column_usage = [True, False] # Columns to use
nInstrPerCol = 1 # Number of instructions per column
imem_add_start = 0 # Start address on imem for this kernel
srf_spm_addres = 0 # Line of the SPM with the initial data for the SRF
version = ""

### FFT

In [ ]:
# Add to KMEM the word and decode it
kmem = KMEM()

kmem_pos_1 = 1
kmem_word_1 = 0x18026

kmem_pos_2 = 2
kmem_word_2 = 0x393b0

kmem.imem.set_word(kmem_word_1, kmem_pos_1)
kmem.imem.set_word(kmem_word_2, kmem_pos_2)
print("Kernel 1")
kmem.imem.get_kernel_info(kmem_pos_1)
print("Kernel 2")
kmem.imem.get_kernel_info(kmem_pos_2)

In [ ]:
# --------------------------------------------
#               KERNEL CONFIGURATION
# --------------------------------------------
kernel_path = './kernels/fft/' 
kernel_number = 1 # Kernel number (from 1 to 15)
column_usage = [True, True] # Columns to use
nInstrPerCol = 39 # Number of instructions per column
imem_add_start = 0 # Start address on imem for this kernel
srf_spm_addres = 0 # Line of the SPM with the initial data for the SRF
version=""

sim = SIMULATOR()
sim.kernel_config(column_usage, nInstrPerCol, imem_add_start, srf_spm_addres, kernel_number)
sim.compileHexToAsm(kernel_path)
sim.compileAsmToHex(kernel_path, kernel_number, version=version)

### MF_Q64_EROSION

In [ ]:
# --------------------------------------------
#               KERNEL CONFIGURATION
# --------------------------------------------
kernel_path = './kernels/mf_q64_erosion/' 
kernel_number = 1 # Kernel number (from 1 to 15)
column_usage = [True, False] # Columns to use
nInstrPerCol = 44 # Number of instructions per column
imem_add_start = 0 # Start address on imem for this kernel
srf_spm_addres = 0 # Line of the SPM with the initial data for the SRF
version=""

sim = SIMULATOR()
sim.kernel_config(column_usage, nInstrPerCol, imem_add_start, srf_spm_addres, kernel_number)
sim.compileHexToAsm(kernel_path)
sim.compileAsmToHex(kernel_path, kernel_number, version=version)